# CuratorAI: A Personalized Art & Museum Discovery Engine
***Building an AI-Powered Museum Guide***

# Table of Contents
1. [Overview 🎯](##overview)
2. [Data Import 📥](##data-import)
3. [Pre-Processing 🧹](##pre-processing)
4. [Exploratory Data Analysis 🔍](##exploratory-data-analysis)
5. [Feature Engineering ⚙️](##feature-engineering)
6. [Model Building & Training 🤖](##model-building--training)
7. [Model Evaluation & Validation 📊](##model-evaluation--validation)
8. [Model Deployment 💾](##model-deployment)
9. [Conclusion & Next Steps 📝](##conclusion--next-steps)


## Overview

**The Art Discovery Problem**

> **"I know what I like, but I don't know how to find more of it."**

This simple frustration echoes through the grand halls of museums worldwide. Visitors stand before masterpieces feeling overwhelmed by choice, unsure where to turn next in collections spanning 5,000 years of human creativity. The Metropolitan Museum of Art alone houses over 470,000 artworks—enough to spend 30 seconds on each piece for 163 consecutive days.


**The Business Challenge**

Museums face a critical engagement problem:
- <u>Visitor Overwhelm</u>: Too many choices lead to decision paralysis
- <u>Personalization Gap</u>: One-size-fits-all audio guides and maps
- <u>Discovery Barriers</u>: Visitors struggle to find artworks that resonate with their personal tastes
- <u>Digital Engagement</u>: Online collections remain underutilized without intelligent navigation


**Solution: CuratorAI**

CuratorAI is an intelligent recommendation system that transforms art discovery from being overwhelming to personalized. By analyzing the textual artworks, their titles, artists, cultural contexts, and historical periods, building bridges between what you love and what you'll love next.

Think of it as: 
> **"The Netflix for art, the Spotify for sculptures, the personal curator in your pocket."**


**Technical Approach**
- <u>Content-Based Filtering</u>: Using TF-IDF and Nearest Neighbors to find similar artworks
- <u>Feature Engineering</u>: Transforming textual metadata into meaningful numerical representations
- <u>End-to-End Pipeline</u>: From raw data to deployed web application
- <u>Real-World Constraints</u>: Working within The Met's Open Access data policies


**Value Proposition**

For **art lovers**: Discover hidden gems and personal connections across centuries
For **museums**: Increase engagement, visit duration, and digital interaction
For **educators**: Create thematic learning journeys through visual culture

**Let's begin our journey through data, art, and machine learning—transforming how the world discovers beauty.**

## Data Import

In [2]:
# Data & ML
import pandas as pd
import numpy as np
import scipy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import joblib

# API & Images
import requests
import json
from PIL import Image
from io import BytesIO
import time

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Utilities
import os
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Loading dataset
df = pd.read_csv('/Users/Rosella/Library/Mobile Documents/com~apple~CloudDocs/Personal Projects/Production-ML-Portafolio/CuratorAI/MetObjects.csv')

# General info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484956 entries, 0 to 484955
Data columns (total 54 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Object Number            484956 non-null  object 
 1   Is Highlight             484956 non-null  bool   
 2   Is Timeline Work         484956 non-null  bool   
 3   Is Public Domain         484956 non-null  bool   
 4   Object ID                484956 non-null  int64  
 5   Gallery Number           49541 non-null   object 
 6   Department               484956 non-null  object 
 7   AccessionYear            481094 non-null  object 
 8   Object Name              482690 non-null  object 
 9   Title                    456153 non-null  object 
 10  Culture                  208190 non-null  object 
 11  Period                   91143 non-null   object 
 12  Dynasty                  23201 non-null   object 
 13  Reign                    11236 non-null   object 
 14  Port

In [4]:
# Data columns
print(f'Rows and columns: {df.shape}')

# First 10 rows
pd.set_option('display.max_columns', None)
print("\nFirst 10 rows with all columns:")
df.head(10)

Rows and columns: (484956, 54)

First 10 rows with all columns:


,Object Number,Is Highlight,Is Timeline Work,Is Public Domain,Object ID,Gallery Number,Department,AccessionYear,Object Name,Title,Culture,Period,Dynasty,Reign,Portfolio,Constituent ID,Artist Role,Artist Prefix,Artist Display Name,Artist Display Bio,Artist Suffix,Artist Alpha Sort,Artist Nationality,Artist Begin Date,Artist End Date,Artist Gender,Artist ULAN URL,Artist Wikidata URL,Object Date,Object Begin Date,Object End Date,Medium,Dimensions,Credit Line,Geography Type,City,State,County,Country,Region,Subregion,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL,Tags Wikidata URL
0,1979.486.1,False,False,False,1,NaN,The American Wing,1979.0,Coin,One-dollar Liberty Head Coin,NaN,NaN,NaN,NaN,NaN,16429,Maker,,James Barton Longacre,"American, Delaware County, Pennsylvania 1794–1...",,"Longacre, James Barton",American,1794,1869,NaN,http://vocab.getty.edu/page/ulan/500011409,https://www.wikidata.org/wiki/Q3806459,1853,1853,1853,Gold,Dimensions unavailable,"Gift of Heinz L. Stoppelmann, 1979",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/1,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
1,1980.264.5,False,False,False,2,NaN,The American Wing,1980.0,Coin,Ten-dollar Liberty Head Coin,NaN,NaN,NaN,NaN,NaN,107,Maker,,Christian Gobrecht,1785–1844,,"Gobrecht, Christian",American,1785,1844,NaN,http://vocab.getty.edu/page/ulan/500077295,https://www.wikidata.org/wiki/Q5109648,1901,1901,1901,Gold,Dimensions unavailable,"Gift of Heinz L. Stoppelmann, 1980",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/2,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
2,67.265.9,False,False,False,3,NaN,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/3,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
3,67.265.10,False,False,False,4,NaN,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/4,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
4,67.265.11,False,False,False,5,NaN,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/5,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
5,67.265.12,False,False,False,6,NaN,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/6,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
6,67.265.13,False,False,False,7,NaN,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/7,NaN,NaN,"Metropolitan Museum of Art, New York, NY",Birds|Coins,http://vocab.getty.edu/page/aat/300266506|http...,https://www.wikidata.org/wiki/Q5113|https://ww...
7,67.265.14,False,False,False,8,NaN,The American Wing

In [5]:
# Systematic assesment
missing_analysis = df.isnull().sum().sort_values(ascending=False)
missing_percentage = (missing_analysis / len(df) * 100)

print("Columns missing values:")
print(missing_percentage)

#  Checking important columns
print('Missing percentage values of relevant columns:')

Columns missing values:
Metadata Date              100.000000
River                       99.568621
State                       99.459539
Locus                       98.449756
County                      98.235098
Reign                       97.683089
Locale                      96.754551
Excavation                  96.582989
Subregion                   95.434019
Dynasty                     95.215855
Rights and Reproduction     94.942015
Portfolio                   94.532700
Region                      93.504565
City                        93.245985
Gallery Number              89.784434
Geography Type              87.636198
Object Wikidata URL         85.740150
Country                     84.327032
Period                      81.205924
Artist Gender               78.042956
Tags                        60.314956
Tags AAT URL                60.314956
Tags Wikidata URL           60.314956
Culture                     57.070332
Artist Wikidata URL         53.768589
Artist ULAN URL           

The initial exploration reveals several data quality challenges that require systematic addressing:

**Data Quality Issues Identified:**
- **Missing Values**: Critical fields like `Artist Display Name` (59% missing), `Object Date` (97.2% missing), `Culture` (43% missing), require strategic handling.
- **Data Type Mismatches**: Numeric fields stored as objects, date information inconsistently formatted
- **Column Relevance**: Only ~15 of 45 columns directly support the recommendation objective
- **Data Integrity**: Need to verify no duplicate `Object ID` entries and validate image URL availability

**Immediate Action Plan:**
1. Identify the optimal subset of columns for feature engineering and check thier missing values percentaje
2. Develop a missing data strategy (imputation vs. filtering)
3. Validate data types and convert where necessary

## Pre-Processing

In [14]:
# Optimal columns
primary_columns = ['Object ID',
                    'Title', 
                    'Object Name',
                    'Object Date',
                    'Department', 
                    'Classification',
                    'Medium'
                    ]

image_columns = ['Link Resource',
                  'Dimensions',
                  'Object Begin Date',
                  'Object End Date']

# Checking missing values percentaje
def checking_nulls(columns):
    for col in columns:
        complete_percentage = (1 - df[col].isnull().mean()) * 100

        print(f'{col}: {complete_percentage}')

checking_nulls(primary_columns)
print()
checking_nulls(image_columns)

Object ID: 100.0
Title: 94.06069829015415
Object Name: 99.53274111465782
Object Date: 97.2304703931903
Department: 100.0
Classification: 83.76821814762576
Medium: 98.51223616163117

Link Resource: 100.0
Dimensions: 84.5227195869316
Object Begin Date: 100.0
Object End Date: 100.0


The selectes columns are 9 with >83% completeness including Object ID, Title, Department, Medium, and Link Resource for images. The next task is handeling the data type correction, missing value handling, duplicate checks.

In [ ]:
# Creating new dataset with select columns
selected_columns = primary_columns + image_columns
data = df[selected_columns].copy()

# Handeling null values
data['Title'].fillna('Untitled', inplace=True)


## Exploratory Data Analysis

In [8]:
# Graficas de obras por autores, o obras por año DEPENDE DE LOS RESULTADOS DE ANTES Y DE LAS COLUMNAS
# Distribuciones por año y autores
# Algo con estilos de arte como contemporaneo

## Feature Engineering